In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Data loading
### reference:https://www.kaggle.com/laowingkin/netflix-movie-recommendation

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8MB 30.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617626 sha256=4efaf522448c294fbb24c303d60f841664b0efbfbea93b1986b8db4e66c91764
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np  #支援高維度陣列還有矩陣運算
import math
import re  #regular expression
from scipy.sparse import csr_matrix  #Sparse Row matrix : https://zhuanlan.zhihu.com/p/36122299
import matplotlib.pyplot as plt 
import seaborn as sns     #比matplot更高階的繪圖 package
from surprise import Reader, Dataset, SVD    #矩陣分解方法:包含SVD
from surprise.model_selection import cross_validate  #the resulting model is validated on the remaining part of the data
sns.set_style("darkgrid")

In [ ]:
# Skip date
df1 = pd.read_csv('/content/gdrive/My Drive/Mercari/Project_on_resume/combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')


Dataset 1 shape: (24058263, 2)
-Dataset examples-


In [ ]:
len(df1['Rating'])

24058263

## Data cleaning

In [ ]:
df1['Rating'] = df1['Rating'].fillna(0)
df1.head(5)

,Cust_Id,Rating
0,1:,0.0
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [ ]:
n_sample_movies = 101

df2 = df1[df1['Rating'] == 0]
last_idx = df2[:n_sample_movies].index[-1]

print(last_idx)

352871


In [ ]:
df = df1.iloc[:last_idx, :]

print(len(df))

352871


In [ ]:
df.index = np.arange(0,len(df))

print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::500, :])

Full dataset shape: (352871, 2)
-Dataset examples-
        Cust_Id  Rating
0            1:     0.0
500     1033930     3.0
1000    1369550     3.0
1500     842962     4.0
2000     466257     4.0
...         ...     ...
350500  2152004     5.0
351000  1966687     4.0
351500  2143500     3.0
352000   201338     3.0
352500  1638786     3.0

[706 rows x 2 columns]


In [ ]:
df.loc[:,'Movie_Id'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm


In [ ]:
change=0
for i in tqdm(range(len(df['Rating']))):
  if df['Rating'][i]==0 :
    change+=1
    df.loc[i,'Movie_Id'] = change
  else:
    df.loc[i,'Movie_Id'] = change

  0%|          | 0/352871 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
100%|██████████| 352871/352871 [12:19<00:00, 477.05it/s]


In [ ]:
df.head(5)

,Cust_Id,Rating,Movie_Id
0,1:,0.0,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1


In [ ]:
df.to_csv("/content/gdrive/My Drive/Mercari/Project_on_resume/test.txt", header=None, index=None, sep=',', mode='w')

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Mercari/Project_on_resume/test.txt', header = None, names = ['Cust_Id', 'Rating','Movie_Id'], usecols = [0,1,2])

In [ ]:
df

,Cust_Id,Rating,Movie_Id
0,1:,0.0,1
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
...,...,...,...
352866,2192562,1.0,100
352867,443447,4.0,100
352868,935149,3.0,100
352869,428688,2.0,100


In [ ]:
df = df.drop(df[df['Rating']==0].index)
df

,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1
...,...,...,...
352866,2192562,1.0,100
352867,443447,4.0,100
352868,935149,3.0,100
352869,428688,2.0,100


In [ ]:
df.index = np.arange(0,len(df))
df

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1
2,885013,4.0,1
3,30878,4.0,1
4,823519,3.0,1
...,...,...,...
352766,2192562,1.0,100
352767,443447,4.0,100
352768,935149,3.0,100
352769,428688,2.0,100


## Data slicing
-Remove movie with too less reviews (they are relatively not popular)
-Remove customer who give too less reviews (they are relatively less active)
since those unpopular movies and non-active customers still occupy same volumn as those popular movies and active customers in the view of matrix (NaN still occupy space).

In [ ]:
f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

Movie minimum times of review: 1274.0
Customer minimum times of review: 2.0


In [ ]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['Movie_Id'].isin(drop_movie_list)]
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::500, :])

Original Shape: (352771, 3)
After Trim Shape: (327086, 3)
-Data Examples-
        Cust_Id  Rating  Movie_Id
692     1025579     4.0         3
1192    2091987     2.0         3
1692    1279413     5.0         3
2192    1807600     4.0         3
2692    1921572     4.0         3
...         ...     ...       ...
350351  1202191     5.0        97
350851  1485530     4.0        97
351351  1748261     5.0        97
351851  1257454     3.0        97
352351  1487575     5.0        97

[655 rows x 3 columns]


### pivot the data set and put it into a giant matrix

In [ ]:
df_p = pd.pivot_table(df,values='Rating',index='Cust_Id',columns='Movie_Id')

print(df_p.shape)

(185478, 30)


In [ ]:
df_p 

Movie_Id,3,8,16,17,18,24,26,28,30,32,33,44,45,46,47,48,52,55,56,57,58,68,71,76,77,78,79,83,84,97
Cust_Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
999935,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN


### Now we load the movie mapping file:

In [ ]:
df_title = pd.read_csv('/content/gdrive/My Drive/Mercari/Project_on_resume/movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])
df_title.set_index('Movie_Id', inplace = True)   #使用現有列設置DataFrame索引
print (df_title.head(10))

            Year                          Name
Movie_Id                                      
1         2003.0               Dinosaur Planet
2         2004.0    Isle of Man TT 2004 Review
3         1997.0                     Character
4         1994.0  Paula Abdul's Get Up & Dance
5         2004.0      The Rise and Fall of ECW
6         1997.0                          Sick
7         1992.0                         8 Man
8         2004.0    What the #$*! Do We Know!?
9         1991.0      Class of Nuke 'Em High 2
10        2001.0                       Fighter


## Recommend with Collaborative Filtering

In [ ]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']][:], reader)

algo = SVD()


cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=3)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9941  0.9906  0.9955  0.9926  0.9913  0.9928  0.0018  
MAE (testset)     0.7862  0.7848  0.7882  0.7885  0.7884  0.7872  0.0015  
Fit time          16.90   17.16   17.05   17.04   17.09   17.05   0.08    
Test time         0.90    0.54    0.97    0.55    0.94    0.78    0.20    


{'fit_time': (16.9011709690094,
  17.158374309539795,
  17.046199560165405,
  17.039102792739868,
  17.092314958572388),
 'test_mae': array([0.78619894, 0.78482227, 0.78815949, 0.78845434, 0.78842268]),
 'test_rmse': array([0.99406106, 0.99061292, 0.9955435 , 0.9926113 , 0.99131752]),
 'test_time': (0.8968605995178223,
  0.5369632244110107,
  0.9730484485626221,
  0.5467138290405273,
  0.9417998790740967)}

## Start to predict!

In [ ]:
df.head(20)

,Cust_Id,Rating,Movie_Id
692,1025579,4.0,3
693,712664,5.0,3
694,1331154,4.0,3
695,2632461,3.0,3
696,44937,5.0,3
697,656399,4.0,3
698,439011,1.0,3
699,1436762,3.0,3
700,1644750,3.0,3
701,2031561,4.0,3


In [ ]:
user_id = 712664				
df_user = df[(df['Cust_Id']==str(user_id))&(df['Rating']==5)]
df_user = df_user.set_index('Movie_Id')
df_user = df_user.join(df_title)['Name']

print(df_user)

Movie_Id
3       Character
79    The Killing
Name: Name, dtype: object


In [ ]:
user = df_title.copy()
user = user.reset_index()
user = user[~user['Movie_Id'].isin(drop_movie_list)]

# getting full dataset
data = Dataset.load_from_df(df[['Cust_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
algo.fit(trainset)

user['Estimate_Score'] = user['Movie_Id'].apply(lambda x: algo.predict(str(user_id), x).est)

user = user.drop('Movie_Id', axis = 1)

user = user.sort_values('Estimate_Score', ascending=False)
print(user.head(3))

      Year                                               Name  Estimate_Score
67  2004.0                                        Invader Zim        4.108040
32  2000.0                     Aqua Teen Hunger Force: Vol. 1        4.104760
31  2004.0  ABC Primetime: Mel Gibson's The Passion of the...        4.046068
